In [ ]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,TemplateSendMessage,PostbackEvent 
)

from linebot.models.template import (
    ButtonsTemplate,PostbackTemplateAction,MessageTemplateAction,URITemplateAction
)

app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# create a instance for line
# line_bot_api 用來處理消息
line_bot_api = LineBotApi('dEB48QaTDE4AMg6gC6qFagwboz8wvTjqdjN1FiA86UT1w4uIVGar+OkcmRokSPCh9pUWSZsatb/qGSDCVKLEc7rAuecn35bwxPFDEk+IyTJdNZL6jTWwVdJb3QugQCOu+0+Pins8qSdZcWE4yKpVpwdB04t89/1O/w1cDnyilFU=')

# 用來接收外部消息
handler = WebhookHandler('e8f9f9ca2a7697893704f71a43fdce6e')

@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body 驗證封包是否來自Line
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'
    
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        thumbnail_image_url='https://cb721c8b.ap.ngrok.io/images/d1664306.jpg',
        title='Menu',
        text='Please select',
        actions=[
            PostbackTemplateAction(
                label='Say hello!',
                text='肚臍你好嗎?',
                data='想睡覺~~'
            ),
            MessageTemplateAction(
                label='Who are you',
                text='我是肚臍'
            ),
            URITemplateAction(
                label='My youtube.',
                uri='https://www.youtube.com/watch?v=uW14n_D2zSc'
            )
        ]
    )
)
    
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        buttons_template_message)
        
@handler.add(PostbackEvent)
def handle_postback_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.postback.data))        
        
if __name__ == "__main__":
    app.run(host='0.0.0.0')